## First make sure proper necessary installations are made

In [ ]:
!pip install pymssql

#### For Mac users:
You will need to install the following program in the terminal or the notebook will throw an error when importing pymssql.

       brew install freetds

Non Mac users may not need this install at all, but in the event that they do, an ubuntu version can be found here:
    https://packages.ubuntu.com/search?keywords=FreeTDS

## Then run imports and test your connection:

In [20]:
import pandas as pd
import numpy as np
import pymssql

In [21]:
conn = pymssql.connect(host='mss-nfl.cuy9yxiaxoxo.us-west-2.rds.amazonaws.com',user='*******',password='**********',database='nfl')


In [22]:
cur = conn.cursor()
cur.execute('''
USE nfl;
''')

### Run the following querry to make sure that eveything works

In [18]:
query = """
SELECT MAX(year) FROM guest.teams;
"""
cur.execute(query)
cur.fetchall() 

[(2019,)]

## Answering questions:

Once everyone is ready we'll dive into the following questions. The table schemas can be found at the bottom of this notebook, however you may find it easier to pull up the github readme for the project, which also contains these schema tables, in another window, link below:

https://github.com/dougtheeconomist/flag-on-the-play/blob/master/README.md

## The Scenario

Suppose we've been hired to do a consulting project for the Seattle Seahawks; there is a feeling that Seattle is penalized quite a bit and maybe disproportionately to other teams, so we want to utilize our database containing penalty, or flag data, to see if there is reasonable evidence for concern. Basically the coaching staff needs to know how hard to push back on the refs when they feel that a bogus call has been made. 

#### Q1 
Write a querry to return the 5 teams who have averaged the highest number of penalties in the last 4 seasons (2016 through 2019) Your querry should return the team id number, team name(city) and the team's average number of penalties over this period. your query could optionally return these teams average yardage lost to penalties over this timeframe as well. 

Is Seattle even on this list?

In [ ]:
conn.rollback()
query = """
Your code here;
"""
cur.execute(query)
cur.fetchall() 

#### Q1 Solution

<details><summary>
Possible answer:
</summary>
SELECT TOP 5 team_city, AVG(against_count), AVG(agnst_yrds)
    
FROM guest.teams
    
WHERE year > 2015
    
GROUP BY team_city
    
ORDER BY AVG(against_count) DESC;

#### Q2 
Next we want to know if the Seahawks started to draw more penalties once Pete Carroll took over as head coach in 2010?
There are several ways to do this. Your querry should return the average number of penalties per season during the Carroll era as well as the average in years prior to this under a different head coach.

In [ ]:
conn.rollback()
query = """
Your code here;
"""
cur.execute(query)
cur.fetchall() 

#### Q2 Solution

<details><summary>
Possible answer:
</summary>
SELECT coach_name, AVG(against_count)
    
FROM guest.teams
    
WHERE team_city = 'Seattle'
    
GROUP BY coach_name;

#### Q3a
In order to really see if the Seahawks are being unfairly targeted for penalties we want to look at a player who has played on a couple of different teams.

To do this we'll start by looking at Richard Sherman, who was let go by Seattle after 2017 and picked up by San Francisco. 

Write a querry that returns Sherman's average flags per season as a Seahawk and as a 49er. Are the averages different?

Hint: it will be helpful to know that Sherman is listed in the player_name column under 'R.Sherman'

In [ ]:
conn.rollback()
query = """
Your code here;
"""
cur.execute(query)
cur.fetchall() 

#### Q3a Solution

<details><summary>
Possible answer:
</summary>
SELECT AVG(pen_count), team
    
FROM guest.players
    
WHERE player_name = 'R.Sherman'
    
GROUP BY team;

#### Q3b

Now that we've done this with one player, we want to expand the querry to look at ALL players who have been on both Seattle and another team. Write a querry that returns these players average number of penalties for the years they were with Seattle and the average number of penalties for their years on other teams. 

<details><summary>
Hint if you don't know where to start:
</summary>
start by writing a querry to find all of the players for Seattle and a querry to find players who have been on multiple teams, then try to incorporate these into your querry from part a

In [ ]:
conn.rollback()
query = """
Your code here;
"""
cur.execute(query)
cur.fetchall() 

#### Q3b Solution

<details><summary>
Possible answer:
</summary>
SELECT player_name, AVG(pen_count), team
    
FROM guest.players
    
WHERE player_id IN (SELECT player_id
FROM guest.players
WHERE player_id IN (SELECT DISTINCT player_id
FROM guest.players
WHERE team = 'Seattle')
GROUP BY player_id
HAVING COUNT(DISTINCT team) > 1)
    
GROUP BY team, player_name
;

In [12]:
cur.close()  # Close the cursor
conn.close()

guest.teams table

|id 			   | Description                                | Type      |
|------------------|:-------------------------------------------|:----------|
| year      	   | Year of football season                    | INT       |
| team_city 	   | City where team is located                 | VARCHAR   |
| team_id   	   | id number unique to team                   | INT       |
| coach_name       | Name of team coach                         | VARCHAR   |
| coach_id         | id number unique to coach                  | INT       |
| ranking          | Rank of most to least penalized            | INT       |
| games     	   | Games played that season                   | INT       |
| plays            | Number of plays that season                | INT       |
| against_count    | Number of flags against team               | INT       |
| agnst_yrds       | Total yards penalized in season            | INT       |
| ben_count        | Number of flags on opposing team           | INT       |
| ben_yrds     	   | Yards given for opposing flags             | INT       |
| net_count        | Team flags less opposing flags	            | INT       |
| net_yrds         | Yards lost plus yards gained from penalties| INT       |
| total_flags      | Total flags thrown in team's games         | INT       |


guest.players table

|id 	        | Description                                | Type      |
|---------------|:-------------------------------------------|:----------|
| year      	| Year of football season                    | INT       |
| player_name 	| Name of player                             | VARCHAR   |
| player_id   	| id number unique to player                 | INT       |
| position      | Abreviation for player's position          | VARCHAR   |
| team          | City where team is located                 | VARCHAR   |
| team_id       | id number unique to team                   | INT       |
| pen_count     | Number of inforced penalties against player| INT       |
| yards         | Yards lost due to flags on player          | INT       |
| n_flags       | All player penalties, includes declined    | INT       |
| pre_snap      | Flags by player prior to start of play     | INT       |
| per_game      | Number of times penalized per game         | INT       |
| yards_game    | Yards lost from penalites per game         | INT       |
| of_team       | Percentage of teams penalties from player  | INT       |
